In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
%matplotlib inline

In [3]:
# Eli's code on median income

median_income_csv = Path("../Data/household_income_msa_2019.csv")
median_income_df = pd.read_csv(median_income_csv)
median_income_df = median_income_df.transpose() # Transpose the column/headers
new_header = median_income_df.iloc[0] #grab the first row for the header
median_income_df = median_income_df[1:] #take the data less the header row
median_income_df.columns = new_header #set the header row as the df header
median_income_df = median_income_df[['Median income (dollars)','Mean income (dollars)']] # Use only median and mean income data columns
median_income_df.reset_index(inplace=True)
median_income_df[['MSA','extra']] = median_income_df['index'].str.split(",", n = 1, expand=True) # Split MSA from text string
median_income_df[['State','extra2']] = median_income_df['extra'].str.split(n = 1, expand=True) # Split State from remaining text string
median_income_df.drop(columns=['index','extra','extra2'],inplace=True) # Drop unnecessary columns
median_income_df.drop_duplicates(subset=['MSA','State'], inplace=True) # Drop duplicates -- first instance is the median and mean income by MSA
median_income_df

Label,Median income (dollars),Mean income (dollars),MSA,State
0,"61,026","77,948",Aberdeen,WA
8,"54,808","71,517",Abilene,TX
16,"53,865","69,443",Adrian,MI
24,"16,311","25,781",Aguadilla-Isabela,PR
32,"57,158","80,800",Akron,OH
...,...,...,...,...
4104,"69,172","85,972",York-Hanover,PA
4112,"48,558","64,409",Youngstown-Warren-Boardman,OH-PA
4120,"61,307","82,466",Yuba City,CA
4128,"46,419","59,182",Yuma,AZ


In [4]:
#checking NY MSA to compare with other datasets

checkmediandf = median_income_df.loc[median_income_df['MSA'].str.contains('Winston')]
checkmediandf

# has 18 rows of NY state MSA, weirdly enough does not include NYC

Label,Median income (dollars),Mean income (dollars),MSA,State
4056,"52,322","71,320",Winston-Salem,NC


In [5]:
# checking rental data from zillow

rental_csv = Path("../Data/rental_prices_msa.csv")
rental_df = pd.read_csv(rental_csv)

checkrentaldf=rental_df[rental_df['RegionName'].str.contains('Winston')]
checkrentaldf

#Zillow Data has 5 rows of MSA for NY


,RegionID,RegionName,SizeRank,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,...,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09
80,395235,"Winston-Salem, NC",81,1021.0,1023.0,1025.0,1027,1029,1031,1033,...,1244,1248,1252,1256,1261,1265,1269,1273,1278,1282


In [6]:
# checking uscities data for long and lat

longlatdensity_csv = Path("../Data/uscities.csv")
longlatdensity_df = pd.read_csv(longlatdensity_csv)

pd.set_option('display.max_rows', None)


checkuscitiesdf=longlatdensity_df[longlatdensity_df['city'].str.contains('Winston')]
checkuscitiesdf

#checkuscitiesdf = longlatdensity_df.loc[(longlatdensity_df.state_id == "NY")]
#checkuscitiesdf.head()

#This one has 1186 of long latitude for New York

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
102,Winston-Salem,Winston-Salem,NC,North Carolina,37067,Forsyth,36.1029,-80.2611,422269.0,721.0,polygon,False,True,America/New_York,2,27110 27127 27040 27107 27106 27105 27104 2710...,1840015324
6356,Winston,Winston,OR,Oregon,41019,Douglas,43.1200,-123.4242,5511.0,804.0,polygon,False,True,America/Los_Angeles,3,97496,1840021275
23425,Winston,Winston,MO,Missouri,29061,Daviess,39.8699,-94.1416,253.0,316.0,polygon,False,True,America/Chicago,3,64689,1840012412
23585,Winston,Winston,MT,Montana,30007,Broadwater,46.4684,-111.6652,245.0,18.0,polygon,False,False,America/Denver,3,59647,1840018475
25372,Winstonville,Winstonville,MS,Mississippi,28011,Bolivar,33.9123,-90.7529,167.0,218.0,polygon,False,True,America/Chicago,3,38762 38781,1840016748
28047,Winston,Winston,NM,New Mexico,35051,Sierra,33.3461,-107.6488,64.0,58.0,polygon,False,False,America/Denver,3,87943,1840025052
